In [8]:
# %% [markdown]
# # Khai bao thư viện cân thiết

# %%
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss


# %% [markdown]
# # Nhập dữ liệu và định dạng
# 

# %%
def load_prepare_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        
    # Kiểm tra dấu phân tách trong dòng đầu tiên
    if ';' in first_line:
        sep = ';'  # Nếu dòng đầu tiên có dấu chấm phẩy, sử dụng dấu chấm phẩy làm phân tách
    else:
        sep = ','  # Nếu không, sử dụng dấu phẩy mặc định
    
    # Đọc file CSV với sep được xác định
    df = pd.read_csv(file_path, sep=sep)

    # In danh sách các cột để kiểm tra
    print(f"Processing file: {file_path}")
    print("Columns:", df.columns.tolist())
    
    # Xử lý cột 'Price', 'Open', 'High', 'Low' nếu có dấu ngoặc kép và chuyển thành float
    for col in ['Price', 'Open', 'High', 'Low']:
        if col in df.columns:
            # Loại bỏ dấu ngoặc kép và chuyển sang kiểu float
            df[col] = df[col].replace({'"': '', ',': ''}, regex=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Kiểm tra nếu có cột 'Date'
    if 'Date' in df.columns:
        try:
            # Chuyển đổi cột 'Date' về kiểu datetime và đặt làm chỉ mục
            df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce').dt.date
           
        except Exception as e:
            print(f"Error converting 'Date' in {file_path}: {e}")

    # Kiểm tra nếu có các cột thời gian của crypto
    elif {'timeOpen', 'timeClose', 'timeHigh', 'timeLow'}.issubset(df.columns):
        try:
            # Chuyển đổi tất cả các cột thời gian về kiểu datetime và chỉ giữ ngày
            for time_col in ['timeOpen', 'timeClose', 'timeHigh', 'timeLow']:
                if time_col in df.columns:
                    df[time_col] = pd.to_datetime(df[time_col], errors='coerce').dt.date
            
            # Đặt 'timeClose' làm chỉ mục và bỏ các cột thời gian khác
            
            # Loại bỏ các cột thời gian không phải 'timeClose'
            time_columns_to_drop = ['timeOpen', 'timeHigh', 'timeLow']
            df.drop(columns=[col for col in time_columns_to_drop if col in df.columns], inplace=True)
            
            
        except Exception as e:
            print(f"Error converting time columns in {file_path}: {e}")
    
    else:
        print(f"No recognized date/time columns in {file_path}")
    
    return df



# %%
# List 11 bộ dữ liệu 

Bitcoin = load_prepare_data(r'Data\Bitcoin_5_14_2010-7_13_2010_historical_data_coinmarketcap.csv')
SP500 = load_prepare_data(r'Data\S&P 500 Historical Data.csv')
Gold = load_prepare_data(r'Data\XAU_USD Historical Data (1).csv')
Silver = load_prepare_data(r'Data\XAG_USD Historical Data.csv')
Tbond = load_prepare_data(r'Data\United States 10-Year Bond Yield Historical Data.csv')
IMUS = load_prepare_data(r'Data\Dow Jones Islamic Market US Historical Data.csv')
WTI = load_prepare_data(r'Data\WTI_USD Historical Data.csv')
Dollar = load_prepare_data(r"Data\US Dollar Index Historical Data.csv")
Franc = load_prepare_data(r'Data\CHF_USD Historical Data.csv')
Ethereum = load_prepare_data(r'Data\Ethereum_6_10_2015-8_9_2015_historical_data_coinmarketcap.csv')
Tether = load_prepare_data(r'Data\Tether USDt_2_12_2015-4_11_2015_historical_data_coinmarketcap.csv')



# %% [markdown]
# # Phân đoạn dữ liệu theo từng giai đoạn khủng hoảng

# %%
Bitcoin = Bitcoin.rename(columns={'timeClose': 'Date'})
Ethereum = Ethereum.rename(columns={'timeClose': 'Date'})
Tether = Tether.rename(columns={'timeClose': 'Date'})

# %%
Bitcoin = Bitcoin.rename(columns={'close': 'Price'})
Ethereum = Ethereum.rename(columns={'close': 'Price'})
Tether = Tether.rename(columns={'close': 'Price'})

# %%
# Tạo dictionary cho tất cả các tài sản
assets = {
    "SP500": SP500,
    "Gold": Gold,
    "Silver": Silver,
    "Tbond": Tbond,
    "IMUS": IMUS,
    "WTI": WTI,
    "Dollar": Dollar,
    "Franc": Franc,
    "Bitcoin": Bitcoin,
    "Ethereum": Ethereum,
    "Tether": Tether
}


# %%
def convert_to_datetime_index_with_copy(assets, date_column='Date', copy_column_name='Time'):
    for name, df in assets.items():
        if date_column in df.columns:
            try:
                # Chuyển đổi cột 'Date' sang kiểu datetime
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
                
                # Loại bỏ các dòng có giá trị null trong cột 'Date' trước khi sao chép
                df.dropna(subset=[date_column], inplace=True)
                
                # Tạo một cột bản sao 'Time' từ cột 'Date'
                df[copy_column_name] = df[date_column]
                
                # Đặt cột 'Time' làm index
                df.set_index(copy_column_name, inplace=True)
                
                print(f"Đã chuyển '{date_column}' thành datetime và tạo index từ cột '{copy_column_name}' cho bộ dữ liệu {name}")
            except Exception as e:
                print(f"Lỗi khi xử lý bộ dữ liệu {name}: {e}")
        else:
            print(f"Bộ dữ liệu {name} không có cột '{date_column}'")
    return assets

# Sử dụng hàm với tập dữ liệu assets
assets = convert_to_datetime_index_with_copy(assets)

# %%
def segment_data_by_period(assets, periods, date_column='Date'):
    # Initialize a dictionary to store segmented data
    segmented_data = {period_name: {} for period_name in periods}

    for period_name, (start_date, end_date) in periods.items():
        # Convert start_date and end_date to datetime and normalize
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for name, df in assets.items():
            # Convert 'Date' column to datetime and normalize if needed
            if date_column in df.columns:
                df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()

            # Check and print min, max dates to confirm the time range
            print(f"Checking data {name} in {period_name}: {df[date_column].min()} - {df[date_column].max()}")

            if period_name == "Full sample":
                # Generate a full date range with weekdays only (excluding weekends)
                full_date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # 'B' frequency for weekdays
                
                # Create a DataFrame with complete weekday dates and NaN for missing dates
                df_full_sample = pd.DataFrame({date_column: full_date_range})
                if date_column in df.columns:
                    # Merge full date range with the original data on 'Date'
                    df_merged = pd.merge(df_full_sample, df, on=date_column, how='left')
                    segmented_data[period_name][name] = df_merged
                else:
                    print(f"Data {name} does not have {date_column} column, skipping.")

            else:
                # Filter data for other periods (GFC, COVID-19)
                if df[date_column].min() <= end_date and df[date_column].max() >= start_date:
                    try:
                        # Extract data within the range
                        df_segment = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]
                        if not df_segment.empty:
                            segmented_data[period_name][name] = df_segment
                            print(f"Extracted data for {name} in {period_name}")
                        else:
                            print(f"No data for asset {name} in {period_name}")
                    except Exception as e:
                        print(f"Error extracting data for {name} in {period_name}: {e}")
                else:
                    print(f"No data for asset {name} in {period_name}")

    return segmented_data



# %%
gfc_period = ('2008-09-12', '2008-10-10')
covid_period = ('2020-01-20', '2020-08-18')
full_sample_period = ('2002-01-02', '2020-08-18')
periods = {
    "Full sample": full_sample_period,
    "GFC": gfc_period,
    "COVID-19": covid_period
}

# Phân đoạn dữ liệu
segmented_data = segment_data_by_period(assets, periods)

# %%
def check_segmented_data(segmented_data, periods, date_column='Date'):
    # Duyệt qua từng giai đoạn và tài sản đã phân đoạn
    for period_name, assets_in_period in segmented_data.items():
        print(f"Giai đoạn: {period_name}")
        start_date, end_date = periods[period_name]  # Lấy mốc thời gian của từng giai đoạn
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for asset_name, df in assets_in_period.items():
            if not df.empty:  # Kiểm tra nếu DataFrame không rỗng
                print(f"  Tài sản: {asset_name}")
                
                # Kiểm tra nếu cột 'Date' tồn tại
                if date_column in df.columns:
                    # Chuyển cột 'Date' thành datetime và chuẩn hóa
                    df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()
                    
                    # Lấy ngày bắt đầu và kết thúc từ cột 'Date'
                    min_date = df[date_column].min()
                    max_date = df[date_column].max()
                    
                    print(f"    Bắt đầu: {min_date}, Kết thúc: {max_date}")
                    
                    # Kiểm tra xem dữ liệu có nằm trong phạm vi giai đoạn không
                    if min_date >= start_date and max_date <= end_date:
                        print(f"    Dữ liệu hợp lệ trong phạm vi {start_date} đến {end_date}")
                    else:
                        print(f"    Lỗi: Dữ liệu không nằm trong phạm vi {start_date} đến {end_date}")
                else:
                    print(f"    Lỗi: Tài sản {asset_name} không có cột '{date_column}'")
            else:
                print(f"  Tài sản: {asset_name} - Không có dữ liệu trong giai đoạn này")
        print("-" * 40)


# %%
# Kiểm tra dữ liệu đã phân đoạn
check_segmented_data(segmented_data, periods)

# %%
segmented_data

Processing file: Data\Bitcoin_5_14_2010-7_13_2010_historical_data_coinmarketcap.csv
Columns: ['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'timestamp']
Processing file: Data\S&P 500 Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: Data\XAU_USD Historical Data (1).csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: Data\XAG_USD Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: Data\United States 10-Year Bond Yield Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Change %']
Processing file: Data\Dow Jones Islamic Market US Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: Data\WTI_USD Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: Data

C:\Users\Legion\AppData\Local\Temp\ipykernel_12300\2072396968.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()
C:\Users\Legion\AppData\Local\Temp\ipykernel_12300\2072396968.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()
C:\Users\Legion\AppData\Local\Temp\ipykernel_12300\2072396968.py:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

{'Full sample': {'SP500':            Date   Price    Open    High     Low  Vol. Change %
  0    2002-01-02  1154.7  1149.0  1154.7  1136.2   NaN    0.57%
  1    2002-01-03  1165.3  1155.5  1165.3  1154.0   NaN    0.92%
  2    2002-01-04  1172.5  1171.1  1176.5  1163.4   NaN    0.62%
  3    2002-01-07  1164.9  1174.7  1177.0  1163.5   NaN   -0.65%
  4    2002-01-08  1160.7  1165.3  1167.6  1157.5   NaN   -0.36%
  ...         ...     ...     ...     ...     ...   ...      ...
  4855 2020-08-12  3380.3  3355.5  3387.9  3355.5   NaN    1.40%
  4856 2020-08-13  3373.4  3372.9  3387.2  3363.3   NaN   -0.20%
  4857 2020-08-14  3372.8  3368.7  3378.5  3361.6   NaN   -0.02%
  4858 2020-08-17  3382.0  3380.9  3387.6  3379.2   NaN    0.27%
  4859 2020-08-18  3389.8  3387.0  3395.1  3370.2   NaN    0.23%
  
  [4860 rows x 7 columns],
  'Gold':            Date    Price     Open     High      Low  Vol. Change %
  0    2002-01-02   278.85   278.85   278.85   278.85   NaN   -0.04%
  1    2002-01-03   

In [9]:
# Save the 'Dollar' data in the 'Full sample' period to a CSV file
segmented_data['Full sample']['Dollar'].to_csv('Dollar_Full_Sample.csv', index=False)
